# CAICT 風機結冰預測 - Deep Learning

2017/07/12  
http://www.industrial-bigdata.com/competition/competitionAction!showDetail.action?competition.competitionId=1

# **1. Load Data and Modules**

**Load Python modules:**

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import datetime
import pandas as pd
import os
import sys
from PIL import Image
import numpy as np
import lzma
import pickle
from IPython.display import display
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
import shutil
import csv
import sklearn as sk
from sklearn.metrics import precision_score
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from pylab import *
from IPython.core.interactiveshell import InteractiveShell
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

pd.set_option('display.max_columns', None)
InteractiveShell.ast_node_interactivity = "all"

In [2]:
def variable_summaries(var, name):  
    with tf.name_scope('summaries_'+str(name)):  
        mean = tf.reduce_mean(var)  
        tf.summary.scalar('mean', mean)  
        stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))  
        tf.summary.scalar('stddev', stddev)  
        tf.summary.scalar('max', tf.reduce_max(var))  
        tf.summary.scalar('min', tf.reduce_min(var))  
        tf.summary.histogram('histogram', var)  
        
def updateProgress(msg):
    sys.stdout.write('\r')
    sys.stdout.write(msg)
    sys.stdout.flush()
    
def myscore(true_y, pred_y):
    n,p =sk.metrics.confusion_matrix(true_y, pred_y)
    tn = n[0]
    fp = n[1]
    fn = p[0]
    tp = p[1]
    #print('tn:',tn,'fp:',fp,'fn:',fn,'fp:',fp)
    score = 1- 0.5*(fp/(tn+fp))- 0.5*(fn/(fn+tp)) 
    #print('score',score)
    return score, {'tn':tn,'fp':fp,'fn':fn,'tp':tp}


In [3]:
def load_resample_data(id, remove_unknown=True, split_case=20):
    data_resample_15 = pd.read_csv('data/resample_data_15.csv')
    if remove_unknown:
        data_resample_15 = data_resample_15[data_resample_15['label']!=2]
    
    train_resample_15 = data_resample_15[data_resample_15['event']<split_case]
    valid_resample_15 = data_resample_15[data_resample_15['event']>=split_case]
    train_X = train_resample_15.drop(['label','time','group','timestamp','event'], axis=1).values
    train_y = train_resample_15['label'].values.astype(int)
    valid_X = valid_resample_15.drop(['label','time','group','timestamp','event'], axis=1).values
    valid_y = valid_resample_15['label'].values.astype(int)
    train_Y = np.eye(2)[train_y]
    valid_Y = np.eye(2)[valid_y]
    
    train_abnormal = train_resample_15[train_resample_15['label']==1]
    train_normal = train_resample_15[train_resample_15['label']==0]
    train_desc = '正常:{i} ({j:.2f} percent), 結冰:{k:} ({m:.2f} percent), Total:{n}'.format(i=len(train_normal), j=len(train_normal)/len(train_resample_15),k=len(train_abnormal),m=len(train_abnormal)/len(train_resample_15), n=len(train_resample_15))
    
    valid_abnormal = valid_resample_15[valid_resample_15['label']==1]
    valid_normal = valid_resample_15[valid_resample_15['label']==0]
    valid_desc = '正常:{i} ({j:.2f} percent), 結冰:{k:} ({m:.2f} percent), Total:{n}'.format(i=len(valid_normal), j=len(valid_normal)/len(valid_resample_15),k=len(valid_abnormal),m=len(valid_abnormal)/len(valid_resample_15), n=len(valid_resample_15))
    
    return train_X, train_Y, train_y, valid_X, valid_Y, valid_y, train_desc, valid_desc

def load_test1_data(id,remove_unknown=True):
    data_raw_21 = pd.read_csv('data/goodformat_21.csv')
    data_raw_21 = data_raw_21.dropna()
    
    data_X = data_raw_21.drop(['label','time','group','timestamp'], axis=1).values
    data_y = data_raw_21['label'].values.astype(int)
    data_Y = np.eye(2)[data_y]
    
    data_abnormal = data_raw_21[data_raw_21['label']==1]
    data_normal = data_raw_21[data_raw_21['label']==0]
    data_desc = '正常:{i} ({j:.2f} percent), 結冰:{k:} ({m:.2f} percent), Total:{n}'.format(i=len(data_normal), j=len(data_normal)/len(data_raw_21),k=len(data_abnormal),m=len(data_abnormal)/len(data_raw_21), n=len(data_raw_21))
    
    return data_X, data_Y, data_y, data_desc

def load_test2_data():
    test_data = pd.read_csv('data/ice1/test/08/08_data.csv')
    test_timeidx = test_data['time']
    tmp = test_data.drop(['time','group'], axis=1)
    test_X = tmp.values
    return test_X, test_timeidx

    

**Load input data.** 

In [16]:
train_X, train_Y, train_y, valid_X, valid_Y, valid_y, train_desc, valid_desc = load_resample_data(15)
valid_21_X, valid_21_Y, valid_21_y, valid_21_desc= load_test1_data(21)
test_X,test_timeidx = load_test2_data()

** 為了RNN需求, 更改X format **

In [49]:
def rnn_dataset(data, n_steps):
    dataset_new = []
    for idx, _ in enumerate(data):
        start_idx = idx-n_steps+1
        end_idx = idx+1
        if start_idx<0:
            continue
        n=np.concatenate(data[start_idx:end_idx], axis=0)
        dataset_new.append(n)
    dataset_new = np.asarray(dataset_new)
    return dataset_new
    

In [ ]:
n_steps = 100  
train_X_new = rnn_dataset(train_X,n_steps)
valid_X_new = rnn_dataset(valid_X,n_steps)
valid_21_X_new = rnn_dataset(valid_21_X,n_steps)
test_X = rnn_dataset(test_X,n_steps)

In [27]:
train_Y_new=train_Y[n_steps:]
train_y_new=train_y[n_steps:]
valid_Y_new=valid_Y[n_steps:]
valid_y_new=valid_y[n_steps:]
valid_21_Y_new=valid_21_Y[n_steps:]
valid_21_y_new=valid_21_y[n_steps:]
test_timeidx_new=test_timeidx[n_steps:]

**風機15 Training Data 狀態正常(0)/異常(1)比例 **

In [28]:
print(train_desc)

正常:35339 (0.61 percent), 結冰:22179 (0.39 percent), Total:57518


**風機15  Validation Data 狀態正常(0)/異常(1)比例 **

In [29]:
print(valid_desc)

正常:7680 (0.64 percent), 結冰:4315 (0.36 percent), Total:11995


** 風機21 Testing Data 狀態正常(0)/異常(1)比例 **

In [30]:
print(valid_21_desc)

正常:168930 (0.94 percent), 結冰:10638 (0.06 percent), Total:179568


## 3. Deep Learning

### 3.1 Tensorflow 參數定義

In [31]:
# hyperparameters
n_inputs = len(train_X[1]) # 每一行的维度
n_hidden_unins = 13 # 中間RNN的hidden units
n_classes = len(train_Y[1])  # RNN最后的输出類別個数

tf.reset_default_graph()
X =tf.placeholder(tf.float32, [None, n_steps, n_inputs], name="X")
Y_ =tf.placeholder(tf.float32, [None, n_classes], name="Y_")

W = {
    'in': tf.Variable(tf.random_uniform([n_inputs, n_hidden_unins], -1.0, 1.0), name="In_W"), # 输入层到中间层权重矩阵
    'out': tf.Variable(tf.random_uniform([n_hidden_unins, n_classes], -1.0, 1.0), name="Out_W"), # 中间层到输出层的权重
}

variable_summaries(W['in'],'In_W')
variable_summaries(W['out'],'Out_W')
B = {
    'in': tf.Variable(tf.constant(0.1, shape=[n_hidden_unins]), name="In_Bias"), # 偏置
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes]), name="Out_Bias"),
}
variable_summaries(B['in'],'In_Bias')
variable_summaries(B['out'],'OUT_Bias')

#tfdot()  太亂了不看~~~


### 3.2 網路結構 Network Structure

In [32]:
# hidden_layer for input
# X : (128, 28, 28)
with tf.name_scope("inLayer"):
    X_ = tf.reshape(X, [-1, n_inputs])  # [batch_size*28, 28]
    X_in = tf.matmul(X_, W['in']) + B['in'] # (-1, 28) matmul (28, hidden_units) => (-1, hidden_units)
    X_in = tf.reshape(X_in, [-1, n_steps, n_hidden_unins]) # (batch_size, steps, input_size)


In [33]:
with tf.name_scope("RNN_CELL"):
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_unins) 
    rnn_outputs, rnn_states = tf.nn.dynamic_rnn(lstm_cell, X_in, dtype=tf.float32) 
    

In [34]:
with tf.name_scope('outlayer'):
    _pred = tf.matmul(rnn_states[1], W['out']) + B['out']
    pred = tf.nn.softmax(_pred, name="pred")
    #rnn_outputs = tf.unstack(tf.transpose(rnn_outputs, [1,0,2]))  
    #pred = tf.matmul(rnn_outputs[-1], weights['out']) + biases['out'] 
    

### 3.3 誤差函數 Loss Function

In [35]:
with tf.name_scope('accuracy'):
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(Y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=_pred, labels=Y_))
    loss_scar = tf.summary.scalar('loss', loss)
    acc_scar = tf.summary.scalar('accuracy', accuracy)

### 3.4 Optimizer (gradient descent)

In [36]:
lr = 0.001 # learning rate
optimizer = tf.train.AdamOptimizer(lr).minimize(loss)


### 3.5 開始訓練  Start Training

In [37]:
#先創造一個session, 然後記得要init variable
init = tf.global_variables_initializer()
merged_summary_op = tf.summary.merge_all()
sess = tf.InteractiveSession()
sess.run(init)

#if not os.path.exists('tb-log_1'):
#    os.mkdir('tb-log_1')
#summary_writer_train = tf.summary.FileWriter('tb-log1/train',graph=tf.get_default_graph())
#summary_writer_validation = tf.summary.FileWriter('tb-log1/validation',graph=tf.get_default_graph())

In [38]:
epoch = 15
batch_size = 300
total_batch= len(train_X) / batch_size
for ep in range(epoch+1):
    for i in range(int(total_batch)+1):
        rnd_idx = np.random.choice(train_X_new.shape[0], batch_size, replace=False)
        batch_x = train_X_new[rnd_idx]
        batch_y = train_Y_new[rnd_idx]
        batch_x = batch_x.reshape([batch_size, n_steps, n_inputs])
        _, loss_v1, acc_v1= sess.run([optimizer, loss_scar, acc_scar], feed_dict={X: batch_x, Y_:batch_y})
        #summary_writer_train.add_summary(summary, ep * total_batch + i)

    batch_vx = valid_X_new.reshape([-1, n_steps, n_inputs])
    loss_v2, acc_v2= sess.run([loss, accuracy], feed_dict={X: batch_vx , Y_: valid_Y_new})
    #summary_writer_validation.add_summary(summary, ep * total_batch + i)
    updateProgress('epoch:{x0}, batch:{x4}, train loss:{x1:.3f} acc:{x5:.3f}, test loss:{x3:.3f} acc:{x2:.3f}'.format(x0=ep,x2=round(acc_v2,3),x3=round(loss_v2,3),x4=i,x1=round(loss_v1,3), x5=round(acc_v1,3)))
    print()


ValueError: cannot reshape array of size 787800 into shape (300,100,26)

In [43]:
batch_x.shape

(300, 2626)

In [48]:
2626*300

787800

## 4. 預測與準確率評估  Evaluation 

#### Predict Training Data

In [ ]:
# predict all X and get the accuracy
train_X_rs = train_X.reshape([-1, n_steps, n_inputs])
acc_t = accuracy.eval({X: train_X_rs , Y_: test_Y})
print('Accuracy:',acc_t)

pred_t = pd.DataFrame(pred.eval({X: train_X_rs}))
pred_t = pred_t.apply(np.argmax,axis=1)
pd.Series(train_y).value_counts()
train_pred.value_counts()

print("Precision", precision_score(train_y, pred_t))
print("Recall", sk.metrics.recall_score(train_y, pred_t))

s,_=myscore(train_y, pred_t)
print('score',s,_)

#### Predict  Validataion Data

In [ ]:
# predict all X and get the accuracy
valid_X_rs = valid_X.reshape([-1, n_steps, n_inputs])
acc_v = accuracy.eval({X: valid_X_rs , Y_: valid_Y})
print('Accuracy:',acc_v)

pred_v = pd.DataFrame(pred.eval({X: valid_X_rs}))
pred_v = pred_v.apply(np.argmax,axis=1)
pred_v.value_counts()

print("Precision", precision_score(valid_y, pred_v))
print("Recall", sk.metrics.recall_score(valid_y, pred_v))

s,_=myscore(valid_y, pred_v)
print('score',s,_)

** Predict another machine **

In [ ]:
# predict all X and get the accuracy
valid_21_X_rs = valid_21_X.reshape([-1, n_steps, n_inputs])
acc_t = accuracy.eval({X: valid_21_X_rs , Y_: valid_21_Y})
print('Accuracy:',acc_t)
pred_t = pd.DataFrame(pred.eval({X: valid_21_X_rs}))
pred_t = pred_t.apply(np.argmax,axis=1)
pred_t.value_counts()

print("Precision", precision_score(valid_21_y, pred_t))
print("Recall", sk.metrics.recall_score(valid_21_y, pred_t))

s,_=myscore(valid_21_y, pred_t)
print('score',s,_)

#### Predict Test Data

In [ ]:
test_X_rs = test_X.reshape([-1, n_steps, n_inputs])
pred_T = pd.DataFrame(pred.eval({X: test_X}))
pred_T = pred_T.apply(np.argmax,axis=1)
pred_T.value_counts()

## 5. 整理預測結果, 正確上傳格式

In [ ]:
startTime=0
endTIme=0
search_start=True
search_end=False
abnormal_list=[]
for i,v in enumerate(pred_T):
    if (v==1) & (search_start):
        startTime = test_timeidx[i]
        search_end=True
        search_start=False
    if (v==0) & (search_end):
        endTIme=test_timeidx[i]
        search_start=True
        search_end=False
        abnormal_list.append((startTime,test_timeidx[i-1]))
len(abnormal_list)

In [ ]:
output_ans=False
if(output_ans):
    with open('test1_08_results.csv','w') as out:
        csv_out=csv.writer(out)
        csv_out.writerow(['startTime','endTime'])
        for row in abnormal_list:
            dummy = csv_out.writerow(row)

print('done')


## 6. Issue
1. Data Imbalance
 - training batch sample 一半normal一半abnormal的data
 - 用兩台風機訓練兩個model, 取故障預測的聯集 

2. 用Precision & Recall評估好壞, 算出Score


other:
- using weighted examples. Just amplify the per-instance loss by a larger weight when seeing positive examples. If you use online gradient descent, it can be as simple as using a larger learning rate when seeing positive examples.
- A similar and slightly better approach (only if you use stochastic gradient descent) is randomly picking an example in each iteration, where the positive examples have higher probability of being picked.

提交结果历史记录  
参赛队伍:　556 / 参赛人数:　858  
 
|竞赛阶段	 |上传者	 |分数	 |提交日期	 |排名	 |下载|
| ---------| -------- | ------ | ------ | ------ | ------ |
|初赛test1阶段|	孔祥千	|55.28684214|	2017/7/12|	50	|下载|

Note: 7/13
1. cross correlation
2. auto correlation
3. common filter
4. de-train 